In [ ]:
# !pip install ultralytics

In [ ]:
# !pip uninstall torch torchvision torchaudio -y

In [ ]:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128

In [2]:
import yaml
from pathlib import Path

import torch

import ultralytics
from ultralytics import YOLO

In [3]:
!python --version

Python 3.10.14


In [4]:
!nvidia-smi

Tue Oct 28 01:39:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 576.52                 Driver Version: 576.52         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5060 Ti   WDDM  |   00000000:2B:00.0  On |                  N/A |
|  0%   45C    P0             23W /  180W |     317MiB /   8151MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
ultralytics.__version__

'8.3.221'

In [6]:
!conda info --envs

# conda environments:
#
base                     C:\ProgramData\anaconda3
default-python-310    *  C:\ProgramData\anaconda3\envs\default-python-310
default-python-310       c:\ProgramData\anaconda3\envs\default-python-310



In [7]:
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())

if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))

2.9.0+cu128
12.8
True
NVIDIA GeForce RTX 5060 Ti


In [8]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 5060 Ti'

In [43]:
!set PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"

print("Device:", device)

Device: cuda


In [26]:
data_yaml_path = "./data.yaml"

In [27]:
with open(data_yaml_path) as f:
    data = yaml.safe_load(f)

print("Classes:", data['names'])
print("Train images:", len(list(Path(data['train']).rglob("*.jpg"))))
print("Val images:", len(list(Path(data['val']).rglob("*.jpg"))))

Classes: ['wheel', 'fire', 'inside_rail', 'LEP', 'outside_rail', 'person', 'rail_switch', 'rail', 'unknown', 'pole', 'train', 'sign']
Train images: 57
Val images: 38


In [47]:
model = YOLO("yolo11l-seg.pt")

In [49]:
model.train(
    data=data_yaml_path,
    epochs=200,
    imgsz=640,       # меньше пикселей — меньше VRAM
    batch=1,         # 1 для уверенной загрузки
    device=0,
    workers=2,
    optimizer="AdamW",
    pretrained=True,
    cache='ram',
    lr0=0.002,
    patience=30,
    close_mosaic=10,
    warmup_epochs=3,
    half=True        # FP16
)

Ultralytics 8.3.221  Python-3.10.14 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5060 Ti, 8151MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=1, bgr=0.0, box=7.5, cache=ram, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=True, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.002, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11l-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train12, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=30, perspective=0.0, plots=True, pose=1

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([ 0,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001C94EDC74C0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.03

In [50]:
metrics = model.val()

metrics

Ultralytics 8.3.221  Python-3.10.14 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5060 Ti, 8151MiB)
YOLO11l-seg summary (fused): 203 layers, 27,593,844 parameters, 0 gradients, 131.9 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 964.5276.9 MB/s, size: 74.8 KB)
val: Scanning D:\Dev\Projects\cv-yolo-train-segment\dataset\valid\labels.cache... 38 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 38/38  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 0.6it/s 5.1s3.5ss
                   all         38        302      0.296      0.129      0.101     0.0399      0.237     0.0597     0.0392     0.0129
                 wheel         15         20          1          0     0.0507     0.0171          1          0    0.00779    0.00196
           inside_rail         22         41      0.121     0.0777     0.0863     0.0453          0          0    0.00111    0.00028
         

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([ 0,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001C9F5682050>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.03

In [51]:
results = model.predict(source="./dataset/test/images", save=True)


image 1/36 d:\Dev\Projects\cv-yolo-train-segment\dataset\test\images\190227-03_jpg.rf.b7a4f22bc449516740722fdcaf07cd4b.jpg: 640x640 (no detections), 332.4ms
image 2/36 d:\Dev\Projects\cv-yolo-train-segment\dataset\test\images\1_jpg.rf.570bc327085ca0d30f6fb49f6abebb69.jpg: 640x640 1 rail, 1 train, 496.9ms
image 3/36 d:\Dev\Projects\cv-yolo-train-segment\dataset\test\images\2025-03-18T15_57_37_298_jpg.rf.5b75a9f3ad9534c2ffc4c8431a56e715.jpg: 640x640 1 LEP, 1 train, 185.4ms
image 4/36 d:\Dev\Projects\cv-yolo-train-segment\dataset\test\images\2025-03-18T15_57_37_299_jpg.rf.fe3a91219e584804180bf6c80cb6a34d.jpg: 640x640 1 train, 504.8ms
image 5/36 d:\Dev\Projects\cv-yolo-train-segment\dataset\test\images\25_3_jpg.rf.5835e1259ab5863a273471c9bd978f55.jpg: 640x640 6 LEPs, 1 person, 187.2ms
image 6/36 d:\Dev\Projects\cv-yolo-train-segment\dataset\test\images\2_jpg.rf.2cf748c946d93658e4caccac8a43ca65.jpg: 640x640 1 train, 501.6ms
image 7/36 d:\Dev\Projects\cv-yolo-train-segment\dataset\test\imag

In [56]:
results = model.val(weights="runs/segment/train/exp/weights/best.pt")

print(results.box.map, results.seg.map)

SyntaxError: '[31m[1mweights[0m' is not a valid YOLO argument. Similar arguments are i.e. ['weight_decay=0.0005'].

    Arguments received: ['yolo', '--f=c:\\Users\\Artemiy\\AppData\\Roaming\\jupyter\\runtime\\kernel-v3f9e676f0705d4ad948c3cd96cfab0075771ef3e9.json']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of ['classify', 'pose', 'detect', 'segment', 'obb']
                MODE (required) is one of ['export', 'train', 'benchmark', 'track', 'predict', 'val']
                ARGS (optional) are any number of custom 'arg=value' pairs like 'imgsz=320' that override defaults.
                    See all ARGS at https://docs.ultralytics.com/usage/cfg or with 'yolo cfg'

    1. Train a detection model for 10 epochs with an initial learning_rate of 0.01
        yolo train data=coco8.yaml model=yolo11n.pt epochs=10 lr0=0.01

    2. Predict a YouTube video using a pretrained segmentation model at image size 320

In [55]:
model.predict(source="./dataset/test/images", save=True, imgsz=1024)


image 1/36 d:\Dev\Projects\cv-yolo-train-segment\dataset\test\images\190227-03_jpg.rf.b7a4f22bc449516740722fdcaf07cd4b.jpg: 1024x1024 1 train, 302.8ms
image 2/36 d:\Dev\Projects\cv-yolo-train-segment\dataset\test\images\1_jpg.rf.570bc327085ca0d30f6fb49f6abebb69.jpg: 1024x1024 1 train, 360.0ms
image 3/36 d:\Dev\Projects\cv-yolo-train-segment\dataset\test\images\2025-03-18T15_57_37_298_jpg.rf.5b75a9f3ad9534c2ffc4c8431a56e715.jpg: 1024x1024 3 LEPs, 621.4ms
image 4/36 d:\Dev\Projects\cv-yolo-train-segment\dataset\test\images\2025-03-18T15_57_37_299_jpg.rf.fe3a91219e584804180bf6c80cb6a34d.jpg: 1024x1024 3 LEPs, 1 train, 280.3ms
image 5/36 d:\Dev\Projects\cv-yolo-train-segment\dataset\test\images\25_3_jpg.rf.5835e1259ab5863a273471c9bd978f55.jpg: 1024x1024 9 LEPs, 1 person, 1 unknown, 656.8ms
image 6/36 d:\Dev\Projects\cv-yolo-train-segment\dataset\test\images\2_jpg.rf.2cf748c946d93658e4caccac8a43ca65.jpg: 1024x1024 2 trains, 372.5ms
image 7/36 d:\Dev\Projects\cv-yolo-train-segment\dataset\t

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: ultralytics.engine.results.Masks object
 names: {0: 'wheel', 1: 'fire', 2: 'inside_rail', 3: 'LEP', 4: 'outside_rail', 5: 'person', 6: 'rail_switch', 7: 'rail', 8: 'unknown', 9: 'pole', 10: 'train', 11: 'sign'}
 obb: None
 orig_img: array([[[106, 109, 107],
         [ 98, 101,  99],
         [ 94,  97,  95],
         ...,
         [ 36,  47,  45],
         [ 34,  45,  43],
         [ 39,  50,  48]],
 
        [[101, 104, 102],
         [ 91,  94,  92],
         [ 88,  91,  89],
         ...,
         [ 30,  41,  39],
         [ 36,  47,  45],
         [ 45,  56,  54]],
 
        [[ 98, 101,  99],
         [ 84,  87,  85],
         [ 82,  85,  83],
         ...,
         [ 34,  45,  43],
         [ 50,  61,  59],
         [ 60,  71,  69]],
 
        ...,
 
        [[162, 158, 147],
         [160, 156, 145],
         [154, 150, 139],
         ...,
       